In [1]:
import torch
import torch.nn as nn
import torchvision

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import os

from torch.utils.data import Dataset, DataLoader
# from torch.utils.data.sampler import Sampler
import torch.optim as optim
import sys
sys.path.append('../')

from dataset import LbpDataset, train_transforms, val_transforms, test_transforms, collate_fn, get_data
from visualize import visualize
from rcnn_model import fasterrcnn_resnet201_fpn, FastRCNNPredictor
from engine import evaluate
import utils


In [2]:
from torchvision.models.detection.faster_rcnn import fasterrcnn_mobilenet_v3_large_fpn

In [3]:
# model = fasterrcnn_mobilenet_v3_large_fpn(pretrained=False, min_size=1600, max_size=1600)

In [4]:
from train_lbp import get_train_test_list

In [5]:
import easydict 
args = easydict.EasyDict({ "batch_size": 16, 
                          "epochs": 50, 
                          "data": 0, 
                          'lr':0.1,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':5,
                          'workers':2,
                         'output_dir' :'../trained_model/fasterrcnn_mobilenet_v3_large_fpn/'})

In [6]:
df = pd.read_csv('../../data/df.csv')
df.head()
# Data loading code
data_dir = '../../data/df.csv'
train_list, test_list = get_train_test_list(data_dir)
train_dataset = LbpDataset(train_list, transform=train_transforms)
test_dataset = LbpDataset(test_list, transform=val_transforms)  

total 4019 train 3014 test 1005
3014
1005


In [7]:
train_sampler = torch.utils.data.RandomSampler(train_dataset)
test_sampler = torch.utils.data.SequentialSampler(test_dataset)

train_loader = DataLoader(
    train_dataset, batch_size=args.batch_size,
    sampler=train_sampler, num_workers=args.workers,
    collate_fn=utils.collate_fn)

test_loader = DataLoader(
    test_dataset, batch_size=args.batch_size,
    sampler=test_sampler, num_workers=args.workers,
    collate_fn=utils.collate_fn)

In [8]:
num_classes = 2
model = fasterrcnn_mobilenet_v3_large_fpn(pretrained=False, min_size=1024, max_size=1024)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
device = torch.device('cuda')
model.to(device)
print('model is loaded to gpu')

model is loaded to gpu


In [9]:
params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.Adam(params, lr=0.0001)
optimizer = torch.optim.SGD(
       params, lr=0.001, momentum=0.9, weight_decay=2e-5)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 40, 60, 80, 100], 
                                                    gamma=0.5)

In [10]:
# checkpoint = torch.load('../trained_model/model.pt')
# model.load_state_dict(checkpoint['model'])
# optimizer.load_state_dict(checkpoint['optimizer'])
# lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
# epoch = checkpoint['epoch']
# print(epoch)

In [11]:
from train_lbp import train_one_epoch

start_time = time.time()
for epoch in range(120):
    train_one_epoch(model, optimizer, train_loader, device, epoch, 400)
    lr_scheduler.step()
    
    if epoch > 60 and epoch % 5 == 0 :
        if args.output_dir:
            checkpoint = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict(),
                'args': args,
                'epoch': epoch
            }
            
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'model_{}.pth'.format(epoch)))
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'checkpoint.pth'))

    if epoch % 5 == 0 :    
        # evaluate after every epoch
        evaluate(model, test_loader, device=device)    
print('total time is {}'.format(time.time() - start_time))    

Epoch: [0]  [  0/189]  eta: 0:09:35  lr: 0.001000  loss: 1.3638 (1.3638)  loss_classifier: 0.6597 (0.6597)  loss_box_reg: 0.0011 (0.0011)  loss_objectness: 0.6996 (0.6996)  loss_rpn_box_reg: 0.0034 (0.0034)  time: 3.0426  data: 2.5987  max mem: 5388
Epoch: [0]  [188/189]  eta: 0:00:01  lr: 0.001000  loss: 0.1358 (0.2033)  loss_classifier: 0.0357 (0.0613)  loss_box_reg: 0.0121 (0.0124)  loss_objectness: 0.0570 (0.1071)  loss_rpn_box_reg: 0.0238 (0.0225)  time: 1.0689  data: 0.6733  max mem: 5532
Epoch: [0] Total time: 0:03:37 (1.1520 s / it)
creating index...
index created!
Test:  [ 0/63]  eta: 0:02:50  model_time: 0.2208 (0.2208)  evaluator_time: 0.0035 (0.0035)  time: 2.7133  data: 2.4490  max mem: 5532
Test:  [62/63]  eta: 0:00:01  model_time: 0.1978 (0.1981)  evaluator_time: 0.0024 (0.0024)  time: 1.0879  data: 0.8619  max mem: 5532
Test: Total time: 0:01:10 (1.1257 s / it)
Averaged stats: model_time: 0.1978 (0.1981)  evaluator_time: 0.0024 (0.0024)
Accumulating evaluation results..